In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# os.environ["TORCH_COMPILE_DEBUG"]="1"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-6.7b", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of OPTForCausalLM were not initialized from the model checkpoint at facebook/opt-6.7b and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [3]:
torch._dynamo.reset()

# g10backend = G10AnalyzerBackend()
# resnet50_compiled = torch.compile(resnet50)
# resnet50_compiled = resnet50_compiled.to(device)

torch.TORCH_LOGS = "+dynamo,+recompiles"
torch.TORCHDYNAMO_VERBOSE = 1
torch._dynamo.config.capture_dynamic_output_shape_ops = True
torch._dynamo.config.verbose = True
import logging
torch._logging.set_logs(dynamo=logging.INFO, graph_breaks=True, schedule=True,trace_call=True)
model_compiled = torch.compile(
        model,
        options={
            "trace.enabled": True,
        },
)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
LEARNING_RATE = 1e-3
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.CrossEntropyLoss()

In [6]:
input_tensor = model.dummy_inputs["input_ids"].clone().to("cuda")
labels = input_tensor.clone().to("cuda")
targets = input_tensor.clone().to("cuda")

# Pre-train the model
model_compiled.train()

for epoch in range(2):
    optimizer.zero_grad()
    outputs = model_compiled(input_tensor, labels)
    loss = loss_function(outputs.pooler_output, outputs.pooler_output)
    loss.backward()
    optimizer.step()

# Save the pre-trained model
# model.save_pretrained("my_bert_model")

[2024-02-04 14:42:40,942] [0/0] torch._dynamo.symbolic_convert: [INFO] Step 1: torchdynamo start tracing new_forward /home/yuqixue2/miniconda3/lib/python3.11/site-packages/accelerate/hooks.py:159
[2024-02-04 14:42:40,944] [0/0] torch.fx.experimental.symbolic_shapes: [INFO] create_env


[2024-02-04 14:42:40,976] [0/0] torch._dynamo.symbolic_convert.__trace_call: [DEBUG] TRACE inlined call pre_forward from /home/yuqixue2/miniconda3/lib/python3.11/site-packages/accelerate/hooks.py:160 in new_forward (add_hook_to_module.new_forward)
[2024-02-04 14:42:40,976] [0/0] torch._dynamo.symbolic_convert.__trace_call: [DEBUG]         args, kwargs = module._hf_hook.pre_forward(module, *args, **kwargs)
[2024-02-04 14:42:40,976] [0/0] torch._dynamo.symbolic_convert.__trace_call: [DEBUG]                        ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
[2024-02-04 14:42:40,979] [0/0] torch._dynamo.symbolic_convert.__trace_call: [DEBUG] TRACE inlined call find_device from /home/yuqixue2/miniconda3/lib/python3.11/site-packages/accelerate/hooks.py:281 in pre_forward (AlignDevicesHook.pre_forward) (inline depth: 1)
[2024-02-04 14:42:40,979] [0/0] torch._dynamo.symbolic_convert.__trace_call: [DEBUG]             self.input_device = find_device([args, kwargs])
[2024-02-04 14:42:40,

RuntimeError: expected scalar type Half but found Float

In [ ]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)


# bknd = G10AnalyzerBackend()
# torch._dynamo.register_backend(bknd, "g10")
trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=1,#100, 
        max_steps=2,#200, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs',
        torch_compile=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()